# Getting Started with COMPASS

COMPASS (**C**omputational **O**ptics for **M**icro-**P**ixel **A**rray **S**ensor **S**imulation) is a
Python framework for simulating the optical response of image sensor pixels. It provides:

- A **material database** with refractive index models (Cauchy, Sellmeier, tabulated) for common
  semiconductor, dielectric, and color filter materials.
- A **solver-agnostic pixel stack** representation that constructs full 3D pixel structures from
  configuration dictionaries.
- Tools for computing **permittivity grids**, **layer slices**, and **Fresnel reflectance**.

In this tutorial we will:

1. Import and explore the `MaterialDB` -- list materials and plot refractive index curves.
2. Build a `PixelStack` for a 2x2 Bayer pixel with 1.0 um pitch.
3. Inspect the stack: layers, domain size, Bayer map, and photodiodes.
4. Retrieve layer slices and visualize a permittivity grid at 550 nm.
5. Compute Fresnel reflectance at an air/silicon interface.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from compass.core.units import nm_to_um, um_to_nm
from compass.geometry.pixel_stack import PixelStack
from compass.materials.database import MaterialDB

## 1. Explore the Material Database

The `MaterialDB` class is the central registry for optical constants. On construction it
loads a set of built-in materials (air, SiO2, Si3N4, HfO2, TiO2, silicon, tungsten, and
Bayer color filters). You can also register custom materials using the Cauchy, Sellmeier,
constant, or tabulated (CSV) interfaces.

Let us start by listing all available materials.

In [ ]:
db = MaterialDB()
print("Available materials:")
for name in db.list_materials():
    print(f"  - {name}")

### 1a. Refractive index of silicon

Silicon is the most important material in image sensor simulation. Its complex refractive
index `n + ik` determines both the focusing/refraction behaviour (real part `n`) and the
optical absorption (imaginary part `k`). Let us plot both components over the visible and
near-IR range.

In [ ]:
wavelengths = np.linspace(0.38, 1.0, 200)  # um

n_si = np.array([db.get_nk("silicon", wl)[0] for wl in wavelengths])
k_si = np.array([db.get_nk("silicon", wl)[1] for wl in wavelengths])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(um_to_nm(wavelengths), n_si, color="tab:blue", linewidth=2)
ax1.set_xlabel("Wavelength (nm)")
ax1.set_ylabel("n (real part)")
ax1.set_title("Silicon -- Refractive Index n")
ax1.grid(True, alpha=0.3)

ax2.plot(um_to_nm(wavelengths), k_si, color="tab:red", linewidth=2)
ax2.set_xlabel("Wavelength (nm)")
ax2.set_ylabel("k (extinction coefficient)")
ax2.set_title("Silicon -- Extinction Coefficient k")
ax2.grid(True, alpha=0.3)

fig.tight_layout()
plt.show()

### 1b. Dielectric materials: SiO2 and color filters

SiO2 is used as a passivation and planarization layer. The color filters (cf_red, cf_green,
cf_blue) have wavelength-dependent absorption that enables color separation. Let us compare
the refractive indices.

In [ ]:
materials_to_plot = ["sio2", "cf_red", "cf_green", "cf_blue"]
colors = ["grey", "red", "green", "blue"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

for mat_name, color in zip(materials_to_plot, colors):
    n_vals = np.array([db.get_nk(mat_name, wl)[0] for wl in wavelengths])
    k_vals = np.array([db.get_nk(mat_name, wl)[1] for wl in wavelengths])
    ax1.plot(um_to_nm(wavelengths), n_vals, color=color, linewidth=2, label=mat_name)
    ax2.plot(um_to_nm(wavelengths), k_vals, color=color, linewidth=2, label=mat_name)

ax1.set_xlabel("Wavelength (nm)")
ax1.set_ylabel("n")
ax1.set_title("Refractive Index (n)")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel("Wavelength (nm)")
ax2.set_ylabel("k")
ax2.set_title("Extinction Coefficient (k)")
ax2.legend()
ax2.grid(True, alpha=0.3)

fig.tight_layout()
plt.show()

## 2. Build a PixelStack

The `PixelStack` class constructs a complete pixel structure from a configuration dictionary.
The configuration specifies the pitch, unit cell size, Bayer pattern, and layer thicknesses.

We will define a 2x2 RGGB Bayer unit cell with a 1.0 um pixel pitch. The stack is built from
bottom to top: silicon, BARL, color filter, planarization, microlens, air.

In [ ]:
# Define a pixel configuration dictionary
pixel_config = {
    "pitch": 1.0,            # um
    "unit_cell": [2, 2],     # 2x2 Bayer
    "bayer_map": [["R", "G"], ["G", "B"]],
    "layers": {
        "silicon": {
            "thickness": 3.0,
            "material": "silicon",
            "photodiode": {
                "position": [0.0, 0.0, 0.5],
                "size": [0.7, 0.7, 2.0],
            },
        },
        "barl": {
            "layers": [
                {"material": "hfo2", "thickness": 0.02},
                {"material": "sio2", "thickness": 0.03},
            ],
        },
        "color_filter": {
            "thickness": 0.6,
            "pattern": "bayer_rggb",
        },
        "planarization": {
            "thickness": 0.3,
            "material": "sio2",
        },
        "microlens": {
            "enabled": True,
            "height": 0.6,
            "material": "polymer_n1p56",
            "radius_x": 0.48,
            "radius_y": 0.48,
            "profile": {"type": "superellipse", "n": 2.5, "alpha": 1.0},
        },
        "air": {"thickness": 1.0},
    },
}

stack = PixelStack(pixel_config, material_db=db)
print("Pixel stack built successfully!")
print(f"  Pitch: {stack.pitch} um")
print(f"  Unit cell: {stack.unit_cell}")
print(f"  Domain size (Lx, Ly): {stack.domain_size} um")
print(f"  Total height: {stack.total_height:.3f} um")
print(f"  Z range: {stack.z_range}")

## 3. Explore the Stack

### 3a. Layer inventory

Each layer carries a name, z-range, thickness, base material, and a flag indicating whether
it is laterally patterned (e.g., the color filter layer has Bayer-patterned permittivity).

In [ ]:
print(f"{'Layer':<18} {'z_start':>8} {'z_end':>8} {'thick':>7} {'material':<16} {'patterned'}")
print("-" * 75)
for layer in stack.layers:
    print(
        f"{layer.name:<18} {layer.z_start:>8.3f} {layer.z_end:>8.3f} "
        f"{layer.thickness:>7.3f} {layer.base_material:<16} {layer.is_patterned}"
    )

### 3b. Bayer map and photodiodes

The Bayer map defines which color filter sits above each pixel. The photodiode list gives
the physical location and size of each photodiode within the silicon layer.

In [ ]:
print("Bayer map:")
for row in stack.bayer_map:
    print(f"  {row}")

print(f"\nPhotodiodes ({len(stack.photodiodes)} total):")
for pd in stack.photodiodes:
    print(
        f"  pixel {pd.pixel_index}, color={pd.color}, "
        f"pos={pd.position}, size={pd.size}"
    )

## 4. Layer Slices and Permittivity Grids

The `get_layer_slices()` method decomposes the pixel stack into z-wise slices, each
containing a 2D complex permittivity grid `eps(x, y)`. This is the input format required
by RCWA solvers.

Patterned layers (microlens, color filter) are resolved on the spatial grid. The microlens
is staircase-approximated into multiple thin slices.

We will generate slices at 550 nm and plot the real part of permittivity for the color
filter layer.

In [ ]:
wavelength = 0.550  # um (green light)
nx, ny = 128, 128

slices = stack.get_layer_slices(wavelength, nx=nx, ny=ny, n_lens_slices=20)

print(f"Total slices at {um_to_nm(wavelength):.0f} nm: {len(slices)}")
print()
for s in slices:
    eps_mean = np.mean(s.eps_grid)
    print(
        f"  {s.name:<25} z=[{s.z_start:.3f}, {s.z_end:.3f}] um  "
        f"eps_mean = {eps_mean.real:.3f} + {eps_mean.imag:.3f}j"
    )

In [ ]:
# Find the color filter slice and plot its permittivity
cf_slice = None
for s in slices:
    if s.name == "color_filter":
        cf_slice = s
        break

if cf_slice is not None:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    lx, ly = stack.domain_size
    extent = [0, lx, 0, ly]

    im1 = ax1.imshow(
        cf_slice.eps_grid.real, origin="lower", extent=extent,
        cmap="viridis", aspect="equal",
    )
    ax1.set_xlabel("x (um)")
    ax1.set_ylabel("y (um)")
    ax1.set_title(f"Color Filter Layer -- Re(eps) at {um_to_nm(wavelength):.0f} nm")
    plt.colorbar(im1, ax=ax1, label="Re(eps)")

    im2 = ax2.imshow(
        cf_slice.eps_grid.imag, origin="lower", extent=extent,
        cmap="magma", aspect="equal",
    )
    ax2.set_xlabel("x (um)")
    ax2.set_ylabel("y (um)")
    ax2.set_title(f"Color Filter Layer -- Im(eps) at {um_to_nm(wavelength):.0f} nm")
    plt.colorbar(im2, ax=ax2, label="Im(eps)")

    fig.tight_layout()
    plt.show()
else:
    print("Color filter slice not found.")

## 5. Fresnel Reflectance at the Air/Silicon Interface

A significant fraction of light is reflected at the air-silicon boundary due to the large
refractive index contrast. We can compute the normal-incidence Fresnel reflectance:

$$R = \left|\frac{n_1 - n_2}{n_1 + n_2}\right|^2$$

where $n_1$ and $n_2$ are the complex refractive indices of air and silicon, respectively.
This calculation shows why anti-reflection coatings (BARL) are essential in image sensors.

In [ ]:
wavelengths = np.linspace(0.38, 1.0, 300)

reflectance = np.zeros_like(wavelengths)
for i, wl in enumerate(wavelengths):
    n_air, k_air = db.get_nk("air", wl)
    n_si, k_si = db.get_nk("silicon", wl)
    # Complex refractive indices
    N_air = n_air + 1j * k_air
    N_si = n_si + 1j * k_si
    # Fresnel reflectance at normal incidence
    r = (N_air - N_si) / (N_air + N_si)
    reflectance[i] = np.abs(r) ** 2

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(um_to_nm(wavelengths), reflectance * 100, color="tab:purple", linewidth=2)
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Reflectance (%)")
ax.set_title("Fresnel Reflectance at Air / Silicon Interface (Normal Incidence)")
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 80)
fig.tight_layout()
plt.show()

# Print some representative values
for wl_target in [0.45, 0.55, 0.65, 0.85]:
    idx = np.argmin(np.abs(wavelengths - wl_target))
    print(f"  R({um_to_nm(wl_target):.0f} nm) = {reflectance[idx]*100:.1f}%")

## Summary

In this tutorial we covered the foundational COMPASS building blocks:

| Concept | Class / Function | What it does |
|---------|-----------------|---------------|
| Material database | `MaterialDB` | Stores and queries n(lam), k(lam) for all materials |
| Pixel geometry | `PixelStack` | Builds the complete 3D pixel stack from a config dict |
| Layer slices | `PixelStack.get_layer_slices()` | Decomposes the stack into 2D eps(x,y) slices for RCWA |
| Unit conversion | `um_to_nm`, `nm_to_um` | Utility functions in `compass.core.units` |

### Next steps

- **02_material_exploration.ipynb** -- Deep dive into material optical properties, absorption
  depths, color filter transmittance, and custom material registration.
- **03_signal_chain.ipynb** -- End-to-end signal chain simulation from illuminant through
  optics to sensor signal, including white balance and SNR analysis.